## Pokec phase diagram

Compute magnetisation as a function of $\beta$ on the Pokec social network using different methods (MC simulations, full graph mean-field, block graph mean-field). For the former two we will only consider the **most positive** and **most negative** metastable solutions. The latter is used in the evaluation of the different influence strategies in the *make_Pokec_control_eval_data.ipynb* notebook.

Created 28/09/20

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import random
import itertools
import matplotlib

from ising_block_level_influence import N_Block_sbm_class as NBlock
from ising_block_level_influence import projection_simplex as proj
from ising_block_level_influence import mean_field_IIM
from spatial_spin_monte_carlo import spatial_spin_monte_carlo as Spins
import Pokec_processing as  PokProc

from pokec_utils import *

### Read in processed data

This data has been generated using:

1. **make_Pokec_reduced_profiles.ipynb** - which then feeds data into:

2. The pre-processing script **'make_bratislava_graph_and_blocks.py'**.

In [2]:
graph = nx.read_graphml('Data/Bratislava_graph.graphml')
bratislava_profiles = pd.read_csv("Data/bratislava_profiles.csv")
coupling_graph = nx.read_graphml('Data/Bratislava_coupling.graphml')
block_data = pd.read_csv('Data/block_info.csv',converters={'Block' : ast.literal_eval})
mean_block_ages = list(block_data['average_age'])
block_sizes = list(block_data['block_size'])
block_names = list(block_data['block_name'])
block_data['age_group'] = [ 'ages_' + k.split('_')[-1] for k in list(block_data['block_name'])]
block_data.head()

,Unnamed: 0,block_name,block_size,average_age,age_group
0,0,Okolie_ages_1-17,1234,9.0,ages_1-17
1,1,Okolie_ages_18-21,1939,19.5,ages_18-21
2,2,Okolie_ages_22-28,3154,25.0,ages_22-28
3,3,Okolie_ages_29-112,3458,70.5,ages_29-112
4,4,Petrzalka_ages_1-17,1398,9.0,ages_1-17


In [3]:
def linear_field(x : np.ndarray,gradient :float) :
    return gradient*x

#Scale ages to [-1,1]:
rescaled_ages = [ (k-np.mean(mean_block_ages))/(max(mean_block_ages)-min(mean_block_ages)) for k in mean_block_ages ]

beta_c = Spins.crit_beta_sparse(graph)

bratislava_profiles_indices = bratislava_profiles.reset_index()
groups = [ bratislava_profiles_indices.loc[bratislava_profiles_indices['block']==block] for block in block_names]
groups_node_ids = [list(k['index']) for k in groups]

Computing critical temperature


In [4]:
#Seed the random number generators:
seed = 1
random.seed(seed)
np.random.seed(seed)

beta_factor_vals = [(10**k) for k in np.linspace(-1,1.8,25)]
grad_vals = [1.0,5.0,10.0]
samples=100
T = 10000
T_Burn = 300000

phase_diag_data = pd.DataFrame()


for gradient in tqdm.tqdm_notebook(grad_vals) : 
    
    age_field = [linear_field(a,gradient) for a in rescaled_ages ]
    age_field_map = {k:j for k,j in zip(list(block_data['age_group']),age_field)}

    # Block MF setup:
    coupling_graph = nx.convert_node_labels_to_integers(coupling_graph)
    block_graph_ising_system = mean_field_IIM.mean_field_ising_system(coupling_graph,age_field)
    block_graph_ising_system.gamma = 1.0
    block_graph_ising_system.tol = 1E-5
    block_graph_ising_system.max_mf_fp_iterations = 10000
    block_graph_ising_system.mf_fp_init_state = -1.0*np.ones(len(block_sizes))
    block_graph_ising_system.mf_fp_noisy = False

    # Full graph Mf setup
    background_field = np.asarray([age_field_map[k] for k in list(bratislava_profiles['age_group'])])
    relab_graph = nx.relabel.convert_node_labels_to_integers(graph)
    full_graph_ising_system = mean_field_IIM.mean_field_ising_system(relab_graph,background_field)
    full_graph_ising_system.mf_fp_init_state = -1.0*np.ones(len(graph))
    full_graph_ising_system.max_mf_fp_iterations = 10000 
    full_graph_ising_system.notebook = True
    full_graph_ising_system.mf_fp_noisy = True
    
    for beta_f in tqdm.tqdm_notebook( beta_factor_vals )  :
    
        beta = beta_f*beta_c
    
        # Full MF: +VE initial condition
        full_graph_ising_system.mf_fp_init_state = np.ones(len(graph))
        m_full = full_graph_ising_system.mf_magnetization(background_field,beta)
        phase_diag_data=phase_diag_data.append(pd.DataFrame({'Method':['full_mf'],'beta_factor':[beta_f],'field_gradient':[gradient],'mean_mag':[np.mean(m_full)],'init':['+VE']}))
        
        # Full MF: -VE initial condition
        full_graph_ising_system.mf_fp_init_state = -1.0*np.ones(len(graph))
        m_full = full_graph_ising_system.mf_magnetization(background_field,beta)
        phase_diag_data=phase_diag_data.append(pd.DataFrame({'Method':['full_mf'],'beta_factor':[beta_f],'field_gradient':[gradient],'mean_mag':[np.mean(m_full)],'init':['-VE']}))
        
        
        
        # Block MF - multiple random initial conditions for each beta:
        for p in range(samples) : 
            block_graph_ising_system.mf_fp_init_state=np.random.uniform(-1,1,len(block_sizes))
            m_block = block_graph_ising_system.mf_magnetization(age_field,beta)
            # Block level average involves multiplying the means by block sizes:
            block_mean_mag = np.sum([i*j for i,j in zip(m_block,block_sizes)])/float(len(graph))  
            phase_diag_data=phase_diag_data.append(pd.DataFrame({'Method':['block_mf'],'beta_factor':[beta_f],'field_gradient':[gradient],'mean_mag':[block_mean_mag],'init':['rand']}))

        # MC simulations: +VE initial condition
        initial_state = np.ones(len(graph))
        spin_series = Run_MonteCarlo_Average(relab_graph, T, beta_f,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)
        phase_diag_data=phase_diag_data.append(pd.DataFrame({'Method':['MC_simulations'],'beta_factor':[beta_f],'field_gradient':[gradient],'mean_mag':[np.mean(spin_series)],'init':['+VE']}))
        
        # MC simulations: -VE initial condition
        initial_state = -1.0*np.ones(len(graph))
        spin_series = Run_MonteCarlo_Average(relab_graph, T, beta_f,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)
        phase_diag_data=phase_diag_data.append(pd.DataFrame({'Method':['MC_simulations'],'beta_factor':[beta_f],'field_gradient':[gradient],'mean_mag':[np.mean(spin_series)],'init':['-VE']}))
        

/home/matt/anaconda3/lib/python3.7/site-packages/ising_block_level_influence-1.0-py3.7.egg/ising_block_level_influence/mean_field_IIM.py:168: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


MF completed with 3 iterations and diff =8.050865479974586e-06 m = 0.0006571620426707654


MF completed with 3 iterations and diff =8.057713779101479e-06 m = 0.0006567794695646815


MF completed with 4 iterations and diff =6.260667238555976e-07 m = 0.0008625510564259774


MF completed with 4 iterations and diff =6.267511607885084e-07 m = 0.0008625111110405592


...

MF completed with 15 iterations and diff =7.685475951424436e-06 m = -0.2954656557258162


MF completed with 15 iterations and diff =1.746030948712729e-07 m = 0.6059746454037233


MF completed with 10 iterations and diff =7.854934681472692e-06 m = -0.29832147266004627


MF completed with 9 iterations and diff =8.345757108951801e-06 m = 0.6123151992580633


MF completed with 8 iterations and diff =9.615066505430647e-06 m = -0.29854948595304753


In [5]:
phase_diag_data.head()

,Method,beta_factor,field_gradient,mean_mag,init
0,full_mf,0.1,1.0,0.000657,+VE
0,full_mf,0.1,1.0,0.000657,-VE
0,block_mf,0.1,1.0,0.000656,rand
0,block_mf,0.1,1.0,0.000656,rand
0,block_mf,0.1,1.0,0.000656,rand


In [6]:
phase_diag_data.to_csv("Data/Pokec_phase_diagram_data.csv")